In [ ]:
import http.client
import json
import psycopg2

# Conexión a la API de la NBA
conn_api = http.client.HTTPSConnection("v2.nba.api-sports.io")
headers = {
    'x-rapidapi-host': "v2.nba.api-sports.io",
    'x-rapidapi-key': "3f41540dfbba32863f7981122084859b"
}
conn_api.request("GET", "/teams/statistics?season=2020&id=1", headers=headers)
res_api = conn_api.getresponse()
data_api = res_api.read()

# Decodificar los datos obtenidos de la API
decoded_data_api = json.loads(data_api.decode("utf-8"))

# Datos de conexión a Redshift
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
dbname = "data-engineer-database"
user = "lucasgfara_coderhouse"
password = "Bs1H2V5S4C"
port = "5439"

# Conexión a Redshift y creación de tabla
conn_redshift = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
cur = conn_redshift.cursor()

# Creación de la tabla en Redshift
cur.execute("""
    CREATE TABLE IF NOT EXISTS nba_team_stats (
        games INT,
        fastBreakPoints INT,
        pointsInPaint INT,
        biggestLead INT,
        secondChancePoints INT,
        pointsOffTurnovers INT,
        longestRun INT,
        points INT,
        fgm INT,
        fga INT,
        fgp FLOAT,
        ftm INT,
        fta INT,
        ftp FLOAT,
        tpm INT,
        tpa INT,
        tpp FLOAT,
        offReb INT,
        defReb INT,
        totReb INT,
        assists INT,
        pFouls INT,
        steals INT,
        turnovers INT,
        blocks INT,
        plusMinus INT
    )
""")

# Insertar datos obtenidos de la API en la tabla
games = decoded_data_api["response"][0]["games"]
fastBreakPoints = decoded_data_api["response"][0]["fastBreakPoints"]
pointsInPaint = decoded_data_api["response"][0]["pointsInPaint"]
biggestLead = decoded_data_api["response"][0]["biggestLead"]
secondChancePoints = decoded_data_api["response"][0]["secondChancePoints"]
pointsOffTurnovers = decoded_data_api["response"][0]["pointsOffTurnovers"]
longestRun = decoded_data_api["response"][0]["longestRun"]
points = decoded_data_api["response"][0]["points"]
fgm = decoded_data_api["response"][0]["fgm"]
fga = decoded_data_api["response"][0]["fga"]
fgp = decoded_data_api["response"][0]["fgp"]
ftm = decoded_data_api["response"][0]["ftm"]
fta = decoded_data_api["response"][0]["fta"]
ftp = decoded_data_api["response"][0]["ftp"]
tpm = decoded_data_api["response"][0]["tpm"]
tpa = decoded_data_api["response"][0]["tpa"]
tpp = decoded_data_api["response"][0]["tpp"]
offReb = decoded_data_api["response"][0]["offReb"]
defReb = decoded_data_api["response"][0]["defReb"]
totReb = decoded_data_api["response"][0]["totReb"]
assists = decoded_data_api["response"][0]["assists"]
pFouls = decoded_data_api["response"][0]["pFouls"]
steals = decoded_data_api["response"][0]["steals"]
turnovers = decoded_data_api["response"][0]["turnovers"]
blocks = decoded_data_api["response"][0]["blocks"]
plusMinus = decoded_data_api["response"][0]["plusMinus"]

cur.execute("""
    INSERT INTO nba_team_stats VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
""", (games, fastBreakPoints, pointsInPaint, biggestLead, secondChancePoints, pointsOffTurnovers, longestRun,
      points, fgm, fga, fgp, ftm, fta, ftp, tpm, tpa, tpp, offReb, defReb, totReb, assists, pFouls, steals,
      turnovers, blocks, plusMinus))

# Confirmar los cambios y cerrar la conexión
conn_redshift.commit()
cur.close()
conn_redshift.close()

print("Tabla creada y datos insertados en Redshift con éxito.")